# Flux.1

Original [Infotrend's CoreAI](https://github.com/Infotrend-Inc/CoreAI) Demo project: https://github.com/Infotrend-Inc/CoreAI-DemoProjects/tree/main/Flux1Schnell

## Description

Load and use the [Flux.1 Schnell](https://huggingface.co/black-forest-labs/FLUX.1-schnell) diffusion model — a lightweight, fast variant of the Flux text-to-image family — for image generation and experimentation in a local environment. 

High-level steps:

1. Install and import the required libraries (e.g., diffusers, torch, transformers, etc.).
2. Configure the runtime device (GPU) for acceleration.
3. Load the Flux1 Schnell model with recommended precision (`torch_dtype=torch.float16`)
4. Run inference with specifed prompts and seeds to generate images

## Pre-requisites

To support features of this notebook with CoreAI, we need to install some libraries that are not pre-installed but are required. 

**Create and Activate the Virtual Environment:**

Open a terminal within the Jupyter notebook (`File -> New -> Terminal`).
Navigate to this project's folder; where we want to set up the environment (where this notebook is located) and run:

```bash
export PROJECT_NAME="Flux1"
export PIP_CACHE_DIR=`pwd`/.cache/pip
mkdir -p $PIP_CACHE_DIR
python -m venv --system-site-packages myvenv
source myvenv/bin/activate
pip install ipykernel
python -m ipykernel install --user --name=${PROJECT_NAME}-myvenv --display-name="Python (${PROJECT_NAME}-myvenv)"
echo ""; echo "Before continuing load the created Python kernel: Python (${PROJECT_NAME}-myvenv)"
```

This will create a local virtual environment to contain installed files to the mounted `/iti` folder (and not modify the container's files).

**Load the Python kernel described above before running the cell below** (it might take a few seconds for the kernel to appear in the list of kernels).

Install the required Libraries (from `requirements.txt`).
The rest of this notebook relies on the proper kernel to be loaded and environment variables to be set. 

In [ ]:
!. ./myvenv/bin/activate && pip install -r requirements.txt

Add `accelerate` to the PATH

In [ ]:
import os
pwd = os.getcwd()
os.environ['PATH'] =  os.path.join(pwd, 'myvenv/bin') + os.pathsep + os.environ['PATH']

! echo $PATH
! which accelerate

**Make sure `HF_HOME` is set BEFORE `transformers` is loaded**

In [ ]:
import os
os.makedirs('HF_HOME', exist_ok=True)
os.environ['HF_HOME'] = 'HF_HOME'

Verify that the `HF_TOKEN` is set before attempting to load the model.

By default, the notebook will hide dot files (i.e. `.env`).
Edit the `env.example` file included in this notebook to reflect needed API keys and rename the file `.env` from a "Terminal".

In [ ]:
# Load environment configuration file
# This sets up the basic structure for API credentials

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if 'HF_TOKEN' not in os.environ:
    printf("No HF_TOKEN set, will not be able to download the model")
    exit(1)

hf_token=os.environ['HF_TOKEN']

## Running Flux1

### Obtain model

In [ ]:
import torch
from diffusers import FluxPipeline

In [ ]:
# Free up some memory before testing
import gc

# check memory
print(torch.cuda.memory_allocated())

gc.collect()
torch.cuda.empty_cache()

# check memory again
print(torch.cuda.memory_allocated())

In [ ]:
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
pipe.enable_sequential_cpu_offload()
torch.cuda.empty_cache()

### Image generation

Modify the `prompt`, `seed` and `width` and `height` as required.

In [ ]:
from IPython.display import Image, display

import os
os.makedirs('out', exist_ok=True)

prompts = [
    ["An ocean reflection on starry night with the word JetStream in the sky with whales and dolpins in the ocean", 798],
    ["An galaxy with the word OpenInfra surrounded by stars", 339],
    ["A floating rover with wheels on a distant planet's landscape moving toward the setting sun. The word Exosphere rising from the horizon", 412],
    ["\"Happy 15th Birthday OpenStack\" with party balloons", 12],
    ["\"Happy 10th Birthday Scientific SIG\" on a complex neural network with activated neurons", 682]
]


width = 1280
height = 720

d = 1
for prompt, seed in prompts:
    image = pipe(
        prompt,
        guidance_scale=0.0,
        num_inference_steps=4,
        max_sequence_length=256,
        height=height,
        width=width,
        generator=torch.Generator("cpu").manual_seed(seed)
    ).images[0]
    fname = f"out/flux-{d}.png"
    image.save(fname)

    print(f"Prompt: {prompt} [seed: {seed}]")
    display(Image(filename=fname))
    d += 1
